In [1]:
%%capture
%pip install - r requirement.txt

In [2]:
import numpy as np
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpMinimize, LpBinary, LpInteger, LpStatus, value, PulpSolverError, PULP_CBC_CMD
import pulp as pl
import time
import heapq
import json
import psycopg2
import psycopg2.extras
import requests
import warnings
import sys
import uuid

debug = 1  # debug mode
TIMEOUT = 500 # timeout 
if debug:
    pd.set_option('display.max_columns', 500)  # show all columns
    
numLimit = 5

if sys.version_info[0:2] != (3, 6):
    warnings.warn('Please use Python3.6', UserWarning)

### INPUT

In [3]:
with open("./parameterDemo1.json") as f:
    paramDict = json.load(f)

In [4]:
queryID = paramDict['query_id']
initialQuery = paramDict['initial_query']
# TODO: link to database system
param = paramDict['secondary_query']
NbvCost = param['prefer']['NBVOrCost']
maxOrMin = param['prefer']['maxOrMin']

fleetAgeLowBound = [None for _ in range(numLimit)]
fleetAgeUpBound = [None for _ in range(numLimit)]
fleetAgeLimit = [None for _ in range(numLimit)]
fleetAgeGeq = [None for _ in range(numLimit)]
weightedAgeLowBound = [None for _ in range(numLimit)]
weightedAgeUpBound = [None for _ in range(numLimit)]
weightedAgeLimit = [None for _ in range(numLimit)]
weightedAgeGeq = [None for _ in range(numLimit)]
lesseeType = [None for _ in range(numLimit)]
lesseeLimit = [None for _ in range(numLimit)]
lesseeGeq = [None for _ in range(numLimit)]
productType = [None for _ in range(numLimit)]
productLimit = [None for _ in range(numLimit)]
productGeq = [None for _ in range(numLimit)]
contractType = [None for _ in range(numLimit)]
contractLimit = [None for _ in range(numLimit)]
contractGeq = [None for _ in range(numLimit)]

minTotalNbv = param['totalNBVFrom'] if param['totalNBVFrom'] else None
maxTotalNbv = param['totalNBVTo'] if param['totalNBVTo'] else None

minTotalCost = param['totalCostFrom'] if param['totalCostFrom'] else None
maxTotalCost = param['totalCostTo'] if param['totalCostTo'] else None

fleetAgeAvgLimit = param['containersAge']['average']['averageWeighedAge'] if param['containersAge']['average']['averageWeighedAge'] else None
fleetAgeAvgGeq = param['containersAge']['average']['symbol']
for i in range(len(param['containersAge']['list'])):
    fleetAgeLowBound[i] = param['containersAge']['list'][i]['containersAgeFrom']
    fleetAgeUpBound[i] = param['containersAge']['list'][i]['containersAgeTo']
    fleetAgeLimit[i] = param['containersAge']['list'][i]['%'] / 100
    fleetAgeGeq[i] = param['containersAge']['list'][i]['symbol']

weightedAgeAvgLimit = param['weightedAge']['average']['averageWeighedAge'] if param['weightedAge']['average']['averageWeighedAge'] else None
weightedAgeAvgGeq = param['weightedAge']['average']['symbol']
for i in range(len(param['weightedAge']['list'])):
    weightedAgeLowBound[i] = param['weightedAge']['list'][i]['weightedAgeFrom']
    weightedAgeUpBound[i] = param['weightedAge']['list'][i]['weightedAgeTo']
    weightedAgeLimit[i] = param['weightedAge']['list'][i]['%'] / 100
    weightedAgeGeq[i] = param['weightedAge']['list'][i]['symbol']

topLesseeLimit = [param['TopLessee'][0]['%'] / 100 if param['TopLessee'][0]['Top1'] else None,
               param['TopLessee'][1]['%'] / 100 if param['TopLessee'][1]['Top2'] else None,
               param['TopLessee'][2]['%'] / 100 if param['TopLessee'][2]['Top3'] else None]
topLesseeGeq = [param['TopLessee'][0]['symbol'] if param['TopLessee'][0]['Top1'] else None,
             param['TopLessee'][1]['symbol'] if param['TopLessee'][1]['Top2'] else None,
             param['TopLessee'][2]['symbol'] if param['TopLessee'][2]['Top3'] else None]
for i in range(len(param['lessee'])):
    lesseeType[i] = param['lessee'][i]['lessee']
    lesseeLimit[i] = param['lessee'][i]['%'] / 100
    lesseeGeq[i] = param['lessee'][i]['symbol']

OnHireLimit = param['status'][0]['%'] / 100 if param['status'][0]['onhire'] else None
OffHireLimit = param['status'][1]['%'] / 100 if param['status'][1]['offhire'] else None
NoneHireLimit = param['status'][2]['%'] / 100 if param['status'][2]['none'] else None

for i in range(len(param['product'])):
    productType[i] = param['product'][i]['productType']
    productLimit[i] = param['product'][i]['%'] / 100
    productGeq[i] = param['product'][i]['symbol']

for i in range(len(param['contractType'])):
    contractType[i] = param['contractType'][i]['contractType']
    contractLimit[i] = param['contractType'][i]['%'] / 100
    contractGeq[i] = param['contractType'][i]['symbol']

#### Load Data

In [5]:
rawData = pd.read_excel(io='./test_data_with_constraints.xlsb', sheet_name='数据', engine='pyxlsb')

In [6]:
data = rawData[['Unit Id Fz', 'Contract Num', 'Cost', 'Product', 'Contract Cust Id', 'Contract Lease Type', 'Nbv', 'Billing Status Fz', 'Fleet Year Fz', 'Age x CEU']].copy()
data.columns = ['unit_id', 'contract_num', 'cost', 'product', 'customer', 'contract', 'nbv', 'billing', 'fleet_year', 'weighted_age']
data.shape

(204504, 10)

### DATA PREPARATION

#### Container Age

Assign to `FleetAge{0}.format(i)`

In [7]:
for i in range(5):
    if fleetAgeLimit[i]:
        column_name = 'FleetAge{0}'.format(i)
        data[column_name] = data['fleet_year'].apply(lambda x: 1 if fleetAgeLowBound[i]<=x<fleetAgeUpBound[i] else 0)

#### Status

Assign new column `Status`

In [8]:
# TODO: check whether we have none
data['OnHireStatus'] = data['billing'].apply(lambda x: 1 if x=='ON' else 0)
data['OffHireStatus'] = data['billing'].apply(lambda x: 1 if x=='OF' else 0)
data['NoneStatus'] = data['billing'].apply(lambda x: 1 if x==' ' else 0)

#### Weighted Age

Assign new column `WeightedAge{0}.format(i)`

In [9]:
for i in range(5):
    if weightedAgeLimit[i]:
        column_name = 'WeightedAge{0}'.format(i)
        data[column_name] = data['weighted_age'].apply(lambda x: 1 if weightedAgeLowBound[i]<=x<weightedAgeUpBound[i] else 0)

#### Product

Assign new column `ProductType{0}.format(i)`

In [10]:
for i in range(5):
    if productLimit[i]:
        column_name = 'ProductType{0}'.format(i)
        data[column_name] = data['product'].apply(lambda x: 1 if x in productType[i] else 0)

#### Lessee

Assign new column `Lessee{0}.format(i)`

In [11]:
# ONE HOT -- all lessee
# no need to consider None
for lesseeName in data['customer'].value_counts().index:
    data[lesseeName] = data['customer'].apply(lambda x: 1 if x==lesseeName else 0)

#### Contract Type

Assign new column `ContractType{0}.format(i)`

In [12]:
for i in range(5):
    if contractLimit[i]:
        column_name = 'ContractType{0}'.format(i)
        data[column_name] = data['contract'].apply(lambda x: 1 if x in contractType[i] else 0)

### Model Part

#### Prepare Parameters

convert to numpy

In [13]:
nbv = data['nbv'].to_numpy()
cost = data['cost'].to_numpy()
onHireStatus = data['OnHireStatus'].to_numpy()
offHireStatus = data['OffHireStatus'].to_numpy()
noneHireStatus = data['NoneStatus'].to_numpy()

fleetAge = []
weightedAge = []
product = []
contract = []
for i in range(numLimit):
    fleetAge.append(data['FleetAge{0}'.format(i)].to_numpy() if fleetAgeLimit[i] else None)
    weightedAge.append(data['WeightedAge{0}'.format(i)].to_numpy() if weightedAgeLimit[i] else None)
    product.append(data['ProductType{0}'.format(i)].to_numpy() if productLimit[i] else None)
    contract.append(data['ContractType{0}'.format(i)].to_numpy() if contractLimit[i] else None)

fleetAgeAvg = data['fleet_year'].to_numpy()
weightedAgeAvg = data['weighted_age'].to_numpy()

lesseeOneHot = {lesseeName: data[lesseeName].to_numpy() for lesseeName in data['customer'].value_counts().index}

# nbv = np.nan_to_num(nbv)
# cost = np.nan_to_num(cost)
# fleetAgeAvg = np.nan_to_num(fleetAgeAvg)
# weightedAgeAvg = np.nan_to_num(weightedAgeAvg)


if debug:
    print('nbv:', nbv.shape)
    print('cost:', cost.shape)
    print('onHireStatus:', onHireStatus.shape)
    print('offHireStatus:', offHireStatus.shape)
    print('noneHireStatus:', noneHireStatus.shape)
    print('fleetAgeAvg:', fleetAgeAvg.shape)
    print('weightedAgeAvg:', weightedAgeAvg.shape)

    print('fleetAge:', fleetAge)
    print('weightedAge:', weightedAge)
    print('productType:', product)
    print('contractType:', contract)

nbv: (204504,)
cost: (204504,)
onHireStatus: (204504,)
offHireStatus: (204504,)
noneHireStatus: (204504,)
fleetAgeAvg: (204504,)
weightedAgeAvg: (204504,)
fleetAge: [None, None, None, None, None]
weightedAge: [None, None, None, None, None]
productType: [array([1, 1, 1, ..., 1, 1, 1]), None, None, None, None]
contractType: [array([1, 1, 1, ..., 1, 1, 1]), None, None, None, None]


#### Define Problem

In [14]:
def SortTop(l, n):
    topN = heapq.nlargest(n, l, key=lambda x:x[1])
    return np.sum(np.stack([lesseeOneHot[topN[i][0]] for i in range(n)]), axis=0)

In [15]:
var = np.array([LpVariable('container_{0}'.format(i), lowBound=0, upBound=1, cat=LpInteger) for i in range(nbv.shape[0])])
prob = LpProblem("MyProblem", LpMaximize if maxOrMin else LpMinimize)

TODO: remove warm up part, fix with warm start.

In [16]:
warmProb = LpProblem("WarmProblem", LpMaximize)
warmProb += lpSum(var * 1)
warmProb.solve(PULP_CBC_CMD(msg = False, timeLimit=1))

1

In [17]:
# objective function 
if NbvCost:
    prob += lpSum(var * nbv)
else:
    prob += lpSum(var * cost)

In [18]:
# constraints
numSelected = lpSum(var) # num of selected containers

# nbv
if maxTotalNbv:
    prob += lpSum(var * nbv) <= maxTotalNbv, "MaxNBV"
    print('Set Max Nbv')
if minTotalNbv:
    prob += lpSum(var * nbv) >= minTotalNbv, "MinNBV"
    print('Set Min Nbv')
    
# cost
if maxTotalCost:
    prob += lpSum(var * cost) <= maxTotalCost, "MaxCost"
    print('Set Max Cost')
if minTotalCost:
    prob += lpSum(var * cost) >= minTotalCost, "MinCost"
    print('Set min Cost')

# status
if OnHireLimit:
    prob += lpSum(var * onHireStatus) >= OnHireLimit * numSelected, "OnHire"
    print('Set Status=ON Rate Limit')
if OffHireLimit:
    prob += lpSum(var * offHireStatus) <= OffHireLimit * numSelected, "OffHire"
    print('Set Status=OF Rate Limit')
if NoneHireLimit:
    prob += lpSum(var * noneHireStatus) <= NoneHireLimit * numSelected, "NoneHire"
    print('Set Status=None Rate Limit') 
  
# container age
if fleetAgeAvgLimit:
    print('Set Container Average Age Limit')
    if fleetAgeAvgGeq:
        prob += lpSum(var * fleetAgeAvg) >= fleetAgeAvgLimit * numSelected, "FleetAgeAvg>"
    else:
        prob += lpSum(var * fleetAgeAvg) <= fleetAgeAvgLimit * numSelected, "FleetAgeAvg<"
for i in range(numLimit):
    if fleetAgeLimit[i]:
        print('Set Container Age Limit', i)
        if fleetAgeGeq[i]:
            prob += lpSum(var * fleetAge[i]) >= fleetAgeLimit[i] * numSelected, "FleetAge{0}>".format(i)
        else:
            prob += lpSum(var * fleetAge[i]) <= fleetAgeLimit[i] * numSelected, "FleetAge{0}<".format(i)

# weighted age
if weightedAgeAvgLimit:
    print('Set Weighted Average Age Limit')
    if weightedAgeAvgGeq:
        prob += lpSum(var * weightedAgeAvg) >= weightedAgeAvgLimit * numSelected, "WeightedAgeAvg>"
    else:
        prob += lpSum(var * weightedAgeAvg) <= weightedAgeAvgLimit * numSelected, "WeightedAgeAvg<"
for i in range(numLimit):
    if weightedAgeLimit[i]:
        print('Set Weighted Age Limit', i)
        if weightedAgeGeq[i]:
            prob += lpSum(var * weightedAge[i]) >= weightedAgeLimit[i] * numSelected, "WeightedAge{0}>".format(i)
        else:
            prob += lpSum(var * weightedAge[i]) <= weightedAgeLimit[i] * numSelected, "WeightedAge{0}<".format(i)

# product
for i in range(numLimit):
    if productLimit[i]:
        print('Set Produdct Limit', i)
        if productGeq[i]:
            prob += lpSum(var * product[i]) >= productLimit[i] * numSelected, "Product{0}>".format(i)
        else:
            prob += lpSum(var * product[i]) <= productLimit[i] * numSelected, "Product{0}<".format(i)

# lessee
for i in range(numLimit):
    if lesseeLimit[i] and lesseeType[i] in lesseeOneHot:
        print('Set Lessee Limit', i)
        if lesseeGeq[i]:
            prob += lpSum(var * lesseeOneHot[lesseeType[i]]) >= lesseeLimit[i] * numSelected, "Lessee{0}>".format(i)
        else:
            prob += lpSum(var * lesseeOneHot[lesseeType[i]]) <= lesseeLimit[i] * numSelected, "Lessee{0}<".format(i)

# TODO: consider the sequence
# find top1, top2, top3
for i in range(min(3, len(lesseeOneHot)), 0, -1):
    if topLesseeLimit[i-1]:
        print('Set Top{0} Limit'.format(i))
        if topLesseeGeq[i-1]:
            prob += lpSum(var * SortTop(list({j: value(lpSum(var * lesseeOneHot[j])) for j in lesseeOneHot.keys()}.items()), i)) >= topLesseeLimit[i-1] * numSelected, "Top{0}>".format(i)
        else:
            prob += lpSum(var * SortTop(list({j: value(lpSum(var * lesseeOneHot[j])) for j in lesseeOneHot.keys()}.items()), i)) <= topLesseeLimit[i-1] * numSelected, "Top{0}<".format(i)

# contract type
for i in range(numLimit):
    if contractLimit[i]:
        print('Set Contract Type Limit, i')
        if contractGeq[i]:
            prob += lpSum(var * contract[i]) >= contractLimit[i] * numSelected, "ContractType{0}>".format(i)
        else:
            prob += lpSum(var * contract[i]) <= contractLimit[i] * numSelected, "ContractType{0}<".format(i)


Set Max Nbv
Set Min Nbv
Set Status=ON Rate Limit
Set Weighted Average Age Limit
Set Produdct Limit 0
Set Top3 Limit
Set Top1 Limit
Set Contract Type Limit, i


In [19]:
solver = PULP_CBC_CMD(msg = True, timeLimit=TIMEOUT)
try:
    prob.solve(solver)
except PulpSolverError:
    print('Nan value is not allowed in model. Please consider data cleaning.')
except Exception as e:
    print(e)

In [20]:
print("==============================================================")
# print(prob)
print("status:", LpStatus[prob.status])
print("==============================================================")
print("target value: ", value(prob.objective))

status: Optimal
target value:  149999999.9909818


In [21]:
# if solution is found
if debug:
    result = np.array([var[i].varValue for i in range(len(var))])
    print(int(sum(result)), '/', len(result), 'containers are selected.')

37759 / 204504 containers are selected.


### Analysis

In [22]:
import collections
collections.Counter(result)

Counter({0.0: 166745, 1.0: 37759})

In [23]:
if debug:    
    result = np.array([var[i].varValue for i in range(len(var))])
    print(int(sum(result)), '/', len(result), 'containers are selected.')
    print('======================================================================')
    print("nbv: {0} between {1} - {2}".format(round(sum(result * nbv), 2), minTotalNbv, maxTotalNbv))
    print("cost: {0} between {1} - {2}".format(round(sum(result * cost), 2), minTotalCost, maxTotalCost))
    print('billing status:')
    if OnHireLimit:
        print('\t onhire {0}, -- {1}'.format(sum(result * onHireStatus)/sum(result), OnHireLimit))
    if OffHireLimit:
        print('\t offhire {0}, -- {1}'.format(sum(result * offHireStatus)/sum(result), OffHireLimit))
    if NoneHireLimit:
        print('\t none {0}, -- {1}'.format(sum(result * noneHireStatus)/sum(result), NoneHireLimit))

    print("container age:")
    if fleetAgeAvgLimit:
        print('\t container average age is {0}, -- {1}'.format(round(sum(result * fleetAgeAvg)/sum(result), 2), fleetAgeAvgLimit))
    for i in range(numLimit):
        if fleetAgeLimit[i]:
            print("\t container age from {0} to {1} is {2}, -- {3}:".format(fleetAgeLowBound[i], fleetAgeUpBound[i], round(sum(result * fleetAge[i])/sum(result), 2), fleetAgeLimit[i]))

    print("weighted age:")
    if weightedAgeAvgLimit:
        print('\t weighted average age is {0}, -- {1}'.format(round(sum(result * weightedAgeAvg)/sum(result), 2), weightedAgeAvgLimit))
    for i in range(numLimit):
        if weightedAgeLimit[i]:
            print("\t weighted age from {0} to {1} is {2}, -- {3}:".format(weightedAgeLowBound[i], weightedAgeUpBound[i], round(sum(result * weightedAge[i])/sum(result), 2), weightedAgeLimit[i]))    

    print("product:")
    for i in range(numLimit):
        if productLimit[i]:
            print("\t product {0} is {1}, -- {2}:".format(productType[i], round(sum(result * product[i])/sum(result), 2), productLimit[i]))    
    
    print("lessee:")
    for i in range(numLimit):
        if lesseeLimit[i]:
            print("\t lessee {0} is {1}, -- {2}:".format(lesseeType[i], round(sum(result * lesseeOneHot[lesseeType[i]])/sum(result), 2), lesseeLimit[i]))    

    print('Top lessee:')
    numLessee = {lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['customer'].value_counts().index}
    sortedLessee = list(numLessee.items())
    top3Lessee = heapq.nlargest(3, sortedLessee, key=lambda x:x[1])
    if topLesseeLimit[0]:
        print('\t top 1 {0} is {1}, -- {2}'.format(top3Lessee[0][0], top3Lessee[0][1] / sum(result), topLesseeLimit[0]))
    if topLesseeLimit[1]:
        if len(top3Lessee) >= 2:
            print('\t top 2 {0} {1} is {2}, -- {3}'.format(top3Lessee[0][0], top3Lessee[1][0], (top3Lessee[0][1] + top3Lessee[1][1])/ sum(result), topLesseeLimit[1]))
        else:
            print('Only one lessee.')
    if topLesseeLimit[2]:
        if len(top3Lessee) >= 3:
            print('\t top 3 {0} {1} {2} is {3}, -- {4}'.format(top3Lessee[0][0], top3Lessee[1][0], top3Lessee[2][0], (top3Lessee[0][1] + top3Lessee[1][1] + top3Lessee[2][1])/ sum(result), topLesseeLimit[2]))
        else:
            print('Only two lessee.')
            
    print("contract type:")
    for i in range(numLimit):
        if contractLimit[i]:
            print("\t contract type {0} is {1}, -- {2}:".format(contractType[i], round(sum(result * contract[i])/sum(result), 2), contractLimit[i])) 


37759 / 204504 containers are selected.
nbv: 149999999.99 between 125000000 - 150000000
cost: 167523900.74 between None - None
billing status:
	 onhire 1.0, -- 0.9
container age:
weighted age:
	 weighted average age is 3.75, -- 3.75
product:
	 product ['D20', 'D40', 'D4H'] is 1.0, -- 0.8:
lessee:
Top lessee:
	 top 1 MSC is 0.449826531422972, -- 0.45
	 top 3 MSC HAPAG CMA is 0.6498583119256336, -- 0.7
contract type:
	 contract type ['LC', 'LT', 'LF', 'LP'] is 1.0, -- 1.0:


### Output

In [24]:
data = data[['unit_id', 'contract_num', 'cost', 'product', 'customer', 'contract', 'nbv', 'billing', 'fleet_year', 'weighted_age']]
data.columns = ['Unit Id Fz', 'Contract Num', 'Cost', 'Product', 'Contract Cust Id', 'Contract Lease Type', 'Nbv', 'Billing Status Fz', 'Fleet Year Fz', 'Age x CEU']
data.shape

(204504, 10)

In [25]:
data.insert(loc=0, column="selected", value=result)
data.to_csv('demo_result.csv')
data.sample()


,selected,Unit Id Fz,Contract Num,Cost,Product,Contract Cust Id,Contract Lease Type,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU
86290,0.0,DFSU2856710,LT-KMTC-13,2704.0,D20,KMTC,LT,1220.344,ON,11.48,11.48


In [26]:
data.sample(2)

,selected,Unit Id Fz,Contract Num,Cost,Product,Contract Cust Id,Contract Lease Type,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU
120323,1.0,DFSU1016216,LT-MSC-36B,2820.0,D20,MSC,LT,-0.09,ON,10.97,10.970
94955,0.0,FFAU3936969,LT-MSC-48,5150.0,D4H,MSC,LT,4828.91,ON,1.11,1.887
